<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C3-Owning_The_Network_With_Scapy/Processing_pcap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Processing pcap__*

Wireshark and other tools like Network Miner are great for interactively exploring packet capture files, but at times you'll want to slice and dice pcap files using Python and Scapy. Some great use cases are generating fuzzing test cases based on captured network traffic or even something as simple as replaying traffic that you have previously captured.

We'll take a slightly different spin on this and attempt to carve out image files from HTTP traffic. With these image files in hand, we will use OpenCV (http://www.opencv.org/), computer vision tool, to attempt to detect images that contain human faces so that we can narrow down images that might be interesting. You can use the previous ARP poisoning scirpt to generate the pcap files, or you could extend the ARP poisoning sniffer to do on-the-fly facial detection of images while the target is browsing.

This example will perform two separate tasks: Carving images out of HTTP traffic and detecting faces in those images. To accommodate this, we'll create two programs so that you can choose to use them separately, depending on the task at hand. You could also use the programs in sequence, as we'll do here. The first program, __recapper.py__, analyzes a pcap file, locates any images that are present in the streams contained in the pcap file, and writes those images to disk. The second program, __detector.py__, analyzes each of those image files to determine if it contains a face. If it does, it writes a new image to disk, adding a box around each face in the image.

Let's get started by dropping in the code necessary to perform the pcap analysis. In the following code, we'll use a __namedtuple__, a Python data structure with fields accessible by attribute lookup. A standard tuple enables you to store a sequence of immutable values; they're almost like lists, except you can't change a tuple's value. The standard tuple uses numerical indexes to access its members
```
point = (1.1, 2.5)
print(point[0], point[1])
```
A __namedtuple__, on the other hand, behaves the same as a regular tuple except that it can access fields through their names. This makes for much more readable code and is also more memory-efficient than a dictionary. The syntax to create a __namedtuple__ requires two arguments: The tuple's name and a space-separated list of field names. For example, say you want to create a data structure called __Point__ with two attributes: __x__ and __y__. You'd define it as follows:
```
Point = namedtuple("Point", ['x', 'y'])
```
Then you could create a __Point__ object named __p__ for example, with the code:
```
p = Point(35, 65)
```
And refer to its attributes just like those of a class: __p.x__ and __p.y__ refer to the __x__ and __y__ attributes of a particular __Point namedtuple__. That is much easier to read than code referring to the index of some item in a regular tuple. In our example, say you create a __namedtuple__ called __Response__ with the following code:
```
Response = namedtuple("Response", ["header", "payload"])
```
Now, instead of referring to an index of a normal tuple, you can use __Response.header__ or __Response.payload__, which is much easier to understand.

Let's use that information in this example. We'll read a __pcap__ file, reconstitute any images that were transferred, and write the images to disk. Open __recapper.py__ and enter the following code:

In [ ]:
from scapy.all import TCP, rdpcap
from collections import namedtuple
import os
import re
import sys
import zlib

OUTDIR = "/root/Desktop/pictures" #[1]
PCAPS = "/root/Downloads"

Response = namedtuple("Response", ["header", "payload"]) #[2]

def get_header(payload): #[3]
    pass

def extract_content(Response, content_name="image"): #[4]
    pass

class Recapper:
    def __init__(self, fname):
        pass

    def get_responses(self): #[5]
        pass

    def write(self, content_name): #[6]
        pass

if __name__ == "__main__":
    pfile = os.path.join(PCAPS, "pcap.pcap")
    recapper = Recapper(pfile)
    recapper.get_responses()
    recapper.write("image")

This is the main skeleton logic of the entire script, and we'll add in the supporting functions shortly. We set up the imports and then specify the location of the directory in which to output the images and the location of the pcap file to read __[1]__. Then we define a __namedtuple__ called __Response__ to have two attributes: The packet __header__ and packet __payload__ __[2]__. We'll create two helper functions to get the packet header __[3]__ and extract the contents __[4]__ that we'll use with the __Recapper__ class we'll define to reconstitute the images present in the packet stream. Besides __\_\_init\_\___, the __Recapper__ class will have two methods: __get_responses__, which will read responses from the pcap file __[5]__, and __write__, which will write image files contained in the responses to the output directory __[6]__.
Let's start filling out this script by writing the __get_header__ function:

In [ ]:
def get_header(payload):
    try:
        header_raw = payload[:payload.index(b"\r\n\r\n") + 2] #[1]
    except ValueError:
        sys.stdout.write('-')
        sys.stdout.flush()
        return None #[2]

    # The regex is fucked up and the tags are interpreted as html, if you're
    # reading from pdf consider this, fuck you colab.
    header = dict(re.findall(r"(?P<name>.*?): (?P<value>.*?)\r\n", header_raw.decode())) #[3]
    if "Content-Type" not in header: #[4]
        return None
    return header

The __get_header__ function takes the raw HTTP traffic and spits out the headers. We extract the header by looking for the portion of the payload that starts at the beginning and ends with a couple of carriage return and newline pairs __[1]__. If the payload doesn't match that pattern, we'll get a __ValueError__, in which case we just write a dash ('-') to the console and return __[2]__.

Otherwise, we create a dictionary (__header__) from the decoded payload, splitting on the colon so that the key is the part before the colon and the value is the part after the colon __[3]__. If the header has no key called __Content-Type__, we return __None__ to indicate that the header doesn't contain the data we want to extract __[4]__. Now let's write a function to extract the content from the response:

In [ ]:
def extract_content(Response, content_name="image"):
    content, content_type = None, None
    if content_name in Response.header["Content-Type"]: #[1]
        content_type = 